In [5]:
import streamlit as st
import pandas as pd
import ollama
import warnings
import pymysql
import time
import torch
import re
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime
from PIL import Image
import plotly.graph_objects as go
import uuid
import html # Import html module for escaping


In [6]:
print("CUDA Version:", torch.version.cuda)
warnings.filterwarnings("ignore")

CUDA Version: 11.8


In [7]:
try:
    db = pymysql.connect(
        host='localhost',
        port=3306,
        user='root',
        passwd='jeminist1029', # Be cautious with hardcoding passwords
        db='zabbix_ver1',
        charset='utf8'
    )
    cursor = db.cursor()
except pymysql.Error as e:
    st.error(f"DB 연결 오류: {e}")
    st.stop() # Stop execution if DB connection fails

In [8]:
def FindSomething(table_name,column_name,something):
  table_name = 'history_uint' if table_name == '*' else table_name
  if column_name=='*' or something=='*':
    # Consider raising an error instead of just warning
    return warnings.warn('Given input include '*'')
  sql = f'''
    SELECT *
    FROM {table_name}
    WHERE {column_name} = {something}
  '''
  return sql

In [9]:
def does_it_exist(conn, table,itemid):
    cur = conn.cursor()
    query = f"""
        SELECT EXISTS(
            SELECT 1
            FROM {table}
            WHERE itemid = %s  # Use parameterized query
        ) AS row_exists;
    """
    try:
        cur.execute(query, (itemid,))
        result = cur.fetchone()[0]
    except pymysql.Error as e:
        st.error(f"DB 오류 (does_it_exist): {e}")
        result = 0 # Assume not exists on error
    finally:
        cur.close()
    return bool(result)

def exist_data(conn, itemid):
    if does_it_exist(conn, 'history',itemid):
        return 1
    elif does_it_exist(conn,'history_uint',itemid):
        return 2
    else:
        return 0


In [10]:
def check_sql_syntax(sql):
    temp_cursor = db.cursor()
    try:
        # Add LIMIT 1 to EXPLAIN to prevent potential long execution
        temp_cursor.execute(f"EXPLAIN {sql.strip().rstrip(';')}") # Basic check
        #temp_cursor.execute(f"EXPLAIN {sql}")
        return True
    except pymysql.Error as e:
        print(f"SQL 문법 오류 감지: {e}") # Log for debugging
        return False
    finally:
        temp_cursor.close()

In [11]:
tmp_sql = """
SELECT itemid, value, FROM_UNIXTIME(clock)
FROM history
WHERE itemid = 40367
AND FROM_UNIXTIME(clock) BETWEEN'2025-02-20 15:02:59' AND '2025-02-21 15:02:59'
"""

In [12]:
check_sql_syntax

<function __main__.check_sql_syntax(sql)>

In [13]:
prompt = {'Users_Question': 'CPU 사용량을 알려줘',
 'Searchable_infomation': {'Zabbix agent ping': '2_40331',
  'Maximum number of open file descriptors': '2_40333',
  'Maximum number of processes': '2_40334',
  'Interface eth0: Inbound packets discarded': '2_42267',
  'Interface eth0: Inbound packets with errors': '2_42269',
  'Interface eth0: Bits received': '2_42271',
  'Interface eth1: Inbound packets discarded': '2_42268',
  'Interface eth1: Inbound packets with errors': '2_42270',
  'Interface eth1: Bits received': '2_42272',
  'Interface eth0: Outbound packets discarded': '2_42273',
  'Interface eth0: Outbound packets with errors': '2_42275',
  'Interface eth0: Bits sent': '2_42277',
  'Interface eth1: Outbound packets discarded': '2_42274',
  'Interface eth1: Outbound packets with errors': '2_42276',
  'Interface eth1: Bits sent': '2_42278',
  '18629 Port Check': '2_55756',
  'Number of processes': '2_40335',
  'Number of running processes': '2_40336',
  'System boot time': '2_40337',
  'Interrupts per second': '1_40338',
  'Load average (15m avg)': '1_40339',
  'Load average (1m avg)': '1_40340',
  'Load average (5m avg)': '1_40341',
  'Number of CPUs': '2_40342',
  'Context switches per second': '1_40343',
  'CPU utilization': '1_40376',
  'CPU guest time': '1_40344',
  'CPU guest nice time': '1_40345',
  'CPU idle time': '1_40346',
  'CPU interrupt time': '1_40347',
  'CPU iowait time': '1_40348',
  'CPU nice time': '1_40349',
  'CPU softirq time': '1_40350',
  'CPU steal time': '1_40351',
  'CPU system time': '1_40352',
  'CPU user time': '1_40353',
  'System local time': '2_40355',
  'Free swap space': '2_40359',
  'Free swap space in %': '1_40360',
  'Total swap space': '2_40361',
  'System uptime': '2_40363',
  'Number of logged in users': '2_40364',
  'vda: Disk average queue size (avgqu-sz)': '1_42266',
  'vdb: Disk average queue size (avgqu-sz)': '1_42259',
  'vda: Disk read request avg waiting time (r_await)': '1_42250',
  'vdb: Disk read request avg waiting time (r_await)': '1_42249',
  'vda: Disk read rate': '1_42265',
  'vdb: Disk read rate': '1_42258',
  'vda: Disk read time (rate)': '1_42264',
  'vdb: Disk read time (rate)': '1_42257',
  'vda: Disk utilization': '1_42263',
  'vdb: Disk utilization': '1_42256',
  'vda: Disk write request avg waiting time (w_await)': '1_42252',
  'vdb: Disk write request avg waiting time (w_await)': '1_42251',
  'vda: Disk write rate': '1_42262',
  'vdb: Disk write rate': '1_42255',
  'vda: Disk write time (rate)': '1_42261',
  'vdb: Disk write time (rate)': '1_42254',
  'Interface eth0: Operational status': '2_42279',
  'Interface eth0: Interface type': '2_42283',
  'Interface eth1: Operational status': '2_42280',
  'Interface eth1: Interface type': '2_42284',
  '/: Free inodes in %': '1_42241',
  '/data: Free inodes in %': '1_42242',
  '/: Space utilization': '1_42243',
  '/: Total space': '2_42245',
  '/: Used space': '2_42247',
  '/data: Space utilization': '1_42244',
  '/data: Total space': '2_42246',
  '/data: Used space': '2_42248',
  'Available memory': '2_40366',
  'Available memory in %': '1_40367',
  'Total memory': '2_40368',
  'Memory utilization': '1_40377',
  'Zabbix agent availability': '2_40369'},
 'Time_now': '2025-04-09 14:38:26'}

In [14]:
#LLM에 질문하기
response = ollama.chat(model='Makint_Query_version5', messages=[
    {
        'role': 'user',
        'content': str(prompt)
    },
])
LLMresult = response['message']['content']
print("^^^^^^^^^^^^^^LLM Result^^^^^^^^^^^^^^")
print(LLMresult)
print("^^^^^^^^^^^^^^LLM Result^^^^^^^^^^^^^^\n")

^^^^^^^^^^^^^^LLM Result^^^^^^^^^^^^^^
SELECT itemid, value, FROM_UNIXTIME(clock)
FROM history
WHERE itemid = 40376
AND FROM_UNIXTIME(clock) BETWEEN '2025-04-09 11:38:26' AND '2025-04-09 14:38:26';
^^^^^^^^^^^^^^LLM Result^^^^^^^^^^^^^^

